### 先處理NER在進行關係分析。

In [ ]:
from langchain.prompts import PromptTemplate

# Define NER Prompt
review_prompt = PromptTemplate(
    input_variables=["json_data"],  # This should match the variable used in the template
    template = """
    請判斷以下 JSON 中每個「主體」與「客體」的類別是否正確，並進行修正：

    ### ** 修正規則如下：
    - 主體的類別是「時間」例如:38年|時間，把主體改成客體，客體改成主體。
    - 主體的類別是「數量」例如:1人|數量，把主體改成客體，客體改成主體。
    - 主體的類別是「貨幣」例如:新台幣|貨幣，把主體改成客體，客體改成主體。
    - 主體的類別是「比例」例如:10%|比例，把主體改成客體，客體改成主體。
    - 客體的類別是「事件」例如:二次大戰|事件，把客體改成主體，把主體改成客體。
    - "我國" → "中華民國"
    - "本局" → "檔案管理局"
    - "[數字]年" → "民國X年"
    - "194x"或"20xx"年份 → "西元XXXX年"
    
    ### ** 修正標記如下：
    - 符合修正的條件時，請直接修改並標註："主體review": "1" 或 "客體review": "1"
    - 並且將有修正的原始類別複製過到："主體original": "警察|組織" 或 "客體original": "警察|組織"
    - 類別正確，標註："主體review": "0" 或 "客體review": "0"
    
    ### ** 輸出要求：
    - **只輸出完整有效的 JSON 陣列**
    - **每個事件為一個獨立 JSON 物件**
    - **請勿輸出多餘文字或 Markdown（例如 ```json ）**
    - 確保所有 key 與 value 使用雙引號，符合 JSON 標準
    - **格式錯誤或非 JSON 視為無效答案**
    
    ### ** 輸入文本: ** 
    {json_data}

    ### **輸出格式範例：**  
    {{
        "relationships": [
            {{
                "事件": [
                    "流氓組織活動"
                ],
                "關係列表": [
                    {{
                        "主體": "警察|職業",
                        "關係": "地點|P159",
                        "客體": "萬華區|地點",
                        "主體review": "1",
                        "主體original": "警察|組織",
                        "客體review": "0"
                    }},
                    {{
                        這裡是下一段，以此類推...
                    }}
                ]
            }}
        ]        
    }}
    
    ## **範例到此結束。**
    """

)


In [2]:
import json
import re


# Clean JSON output by removing markdown
def clean_json_output(raw_output):
    cleaned = re.sub(r"```json\s*([\s\S]*?)\s*```", r"\1", raw_output).strip()
    return cleaned

# Safe JSON loading
# def safe_json_loads(data):
#     if isinstance(data, str):
#         try:
#             return json.loads(data)
#         except json.JSONDecodeError:
#             print("錯誤：無法解析 JSON 字串")
#             return {}
#     return data

def safe_json_loads(data):
    if isinstance(data, str):
        try:
            return json.loads(data)
        except json.JSONDecodeError as e:
            print(f"❌ JSON 解析錯誤：{e.msg}")
            print(f"➡️  發生在第 {e.lineno} 行，第 {e.colno} 列")
            print(f"➡️  錯誤內容：{e.doc.splitlines()[e.lineno - 1]}")
            return {}
    else:
        print("⚠️ 輸入資料型別非字串，無法解析")
    return data

# Process text using NER chain
def process_text(text):
    review = chain.run(text)
    return review
    

def process_json_file(input_file_path):
    """讀取並處理單個 JSON 檔案，並列印處理結果"""
    print(f"處理檔案: {input_file_path}")
    
    try:
        with open(input_file_path, "r", encoding="utf-8") as f:
            data = json.load(f)        

        if "relationships" not in data["result"] or data["result"]["relationships"] is None:
            print(f"檔案 {input_file_path} 中的 'relationships' 欄位為 None 或不存在。")
            return
        
        # 印出 data["result"]["relationships"] 內容
        print("data['result']['relationships'] 的內容:")
        print(json.dumps(data["result"]["relationships"], ensure_ascii=False, indent=4))
    
#
    
    except json.JSONDecodeError as e:
        print(f"無法解析 JSON 文件 {input_file_path}: {e}")
        return  # 當無法解析 JSON 時跳過該文件
    

### 測試處理目錄檔案JSON並列印結果

In [43]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
#os.environ["GROQ_API_KEY"] = ''
#API#2
#s.environ["GROQ_API_KEY"] = ''
#API#3
os.environ["GROQ_API_KEY"] = ''
#API#4
#os.environ["GROQ_API_KEY"] = ''
#API#5
#os.environ["GROQ_API_KEY"] = ''

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    #model_name="llama-3.3-70b-versatile",
    model_name="llama-3.3-70b-versatile",        
    #model_name="llama-3.2-3b-preview",
    #model_name="mixtral-8x7b-32768",
    #model_name="llama-3.2-11b-vision-preview",
    #model_name="deepseek-r1-distill-qwen-32b",
    #model_name="deepseek-r1-distill-llama-70b",
    temperature=0.0,
  #  max_tokens=1000,
)

In [44]:
# 設定基本的 LLM 模型
from langchain.chains import LLMChain
import json
import shutil


# Handle files in the directory
json_dir = "./docs/output/3_ner_re_p/llama_v7/"
done_dir = "./docs/output/3_ner_re_p/llama_v7/done/"
error_dir = "./docs/output/3_ner_re_p/llama_v7/error/"
out_dir = "./docs/output/4_llm_resolution/llama_v7/"

# Ensure output directories exist
#os.makedirs(done_dir, exist_ok=True)
os.makedirs(done_dir, exist_ok=True)
os.makedirs(out_dir, exist_ok=True)

# Assuming you have an LLM object already defined, pass it to the chain
chain = LLMChain(prompt=review_prompt, llm=llm)


# Get all JSON files in the directory

json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for filename in json_files:
    file_path = os.path.join(json_dir, filename)
    
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"✅ JSON檔案: {filename} 處理中...")
        # Get relationships from JSON and process text
        relationships = data["result"]["relationships"]
        #data.get("result", {}).get("relationships", [])
        #print(relationships)
        #clean_json_output(relationships)
        if relationships:
            #print(process_text(relationships))
            processed_relation = safe_json_loads(clean_json_output(process_text(relationships)))
            #processed_relation = safe_json_loads(process_text(relationships))
            #processed_relation = process_text(relationships)
        else:
            print(f"⚠️ {filename} - No relationships found!")
            continue
        
        # Write processed data to output
        if processed_relation:
            out_path = os.path.join(out_dir, filename)
            #print(f"✅ JSON檔案: {filename} 處理中...")
            with open(out_path, "w", encoding="utf-8") as f:
                json.dump(processed_relation, f, indent=1, ensure_ascii=False)            
            print(f"✅ 已處理並存入: {out_path}")
            # Move processed file to done directory
            #shutil.move(file_path, os.path.join(done_dir, filename))
            print(f"📂 已搬移 {filename} 到 {done_dir}")
            
        else:
            print(f"⚠️ JSON 格式輸出錯誤，無法處理檔案，{filename} 已搬移到 {error_dir}")
            #shutil.move(file_path, os.path.join(error_dir, filename))
            #print(f"📂 已搬移 {filename} 到 {error_dir}")
            continue
        
    except json.JSONDecodeError:
        print(f"⚠️ JSON 解析錯誤: {filename}")
    except IOError as e:
        print(f"⚠️ 無法讀取檔案 {filename}，錯誤: {e}")

print("\n📂 全部 JSON 檔案處理完成！")


✅ JSON檔案: 抗戰期間的新四軍事件(85).json 處理中...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01j8j0vjn7emzttpg8qr9y77md` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 97449, Requested 5411. Please try again in 41m10.554s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}